In [1]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm
import torch
import torch.nn as nn
from torch.nn import Parameter
import sys
sys.path.append('.')

from sklearn.metrics import classification_report
from utils import time_since
import traceback

import time

import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torch.utils.data import DataLoader
from torch.autograd import Variable

import math, random
import argparse
from collections import deque
import cPickle as pickle

from fast_jtnn import *
import rdkit

In [2]:
label_col = 10 + 3 
target_unlabeled_percentage = 0.75
early_stop_thresh = 2

load_epoch = 0
alpha = 100 #0.1 * len_unlabelled / len_labelled

hidden_size=56 #450
batch_size=2
latent_size=28
depthT=20
depthG=3
y_size=1

lr=1e-3
clip_norm=50.0
beta=0.0
step_beta=0.002
max_beta=1.0
warmup=40 #40000

total_epoch=5
test_epoch=1
anneal_rate=0.9
anneal_iter=40 #40000
kl_anneal_iter=20 #2000

save_iter=50
print_iter=5

num_workers = 4
has_cuda = torch.cuda.is_available()

save_dir = 'data/qm9/model'
train_folder = 'fast_molvae/qm9_processed/train'
test_folder = 'fast_molvae/qm9_processed/test'
vocab_file = 'data/qm9/vocab.txt'

In [3]:
vocab = [x.strip("\r\n ") for x in open(vocab_file)]
vocab = Vocab(vocab)

In [4]:
model = SemiJTNNVAERegressor(vocab, hidden_size, latent_size, y_size, depthT, depthG, alpha)
print model

SemiJTNNVAERegressor(
  (jtnn): JTNNEncoder(
    (embedding): Embedding(1786, 56)
    (outputNN): Sequential(
      (0): Linear(in_features=112, out_features=56, bias=True)
      (1): ReLU()
    )
    (GRU): GraphGRU(
      (W_z): Linear(in_features=112, out_features=56, bias=True)
      (W_r): Linear(in_features=56, out_features=56, bias=False)
      (U_r): Linear(in_features=56, out_features=56, bias=True)
      (W_h): Linear(in_features=112, out_features=56, bias=True)
    )
  )
  (decoder): JTNNDecoder(
    (embedding): Embedding(1786, 56)
    (W_z): Linear(in_features=112, out_features=56, bias=True)
    (U_r): Linear(in_features=56, out_features=56, bias=False)
    (W_r): Linear(in_features=56, out_features=56, bias=True)
    (W_h): Linear(in_features=112, out_features=56, bias=True)
    (W): Linear(in_features=70, out_features=56, bias=True)
    (U): Linear(in_features=70, out_features=56, bias=True)
    (U_i): Linear(in_features=112, out_features=56, bias=True)
    (W_o): Linea

/home/tsa87/miniconda3/envs/python2/lib/python2.7/site-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [5]:
# model = NNVAE(1024, hidden_size, latent_size, y_size, alpha)
# print model

In [6]:
if has_cuda: model = model.cuda()

In [7]:
for param in model.parameters():
    if param.dim() == 1:
        nn.init.constant_(param, 0)
    else:
        nn.init.xavier_normal_(param)

In [8]:
if load_epoch > 0:
    model.load_state_dict(torch.load(save_dir + "/model.iter-" + str(load_epoch)))
    
print "Model #Params: %dK" % (sum([x.nelement() for x in model.parameters()]) / 1000,)

Model #Params: 392K


In [9]:
optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = lr_scheduler.ExponentialLR(optimizer, anneal_rate)

In [10]:
param_norm = lambda m: math.sqrt(sum([p.norm().item() ** 2 for p in m.parameters()]))
grad_norm = lambda m: math.sqrt(sum([p.grad.norm().item() ** 2 for p in m.parameters() if p.grad is not None]))

In [11]:
# Doing this because 20% of labelled data got lost in test
tracker = IndexTracker(train_folder, label_idx=label_col, label_pct=(1-target_unlabeled_percentage)/0.8)

In [12]:
def train(model, optimizer, scheduler, use_moltree=True, patience=3):
    best_loss = np.inf
    no_improvement = 0
    
    beta = 0
    total_step = load_epoch
    
    start = time.time()
    for epoch in xrange(total_epoch):
        
        preds = np.array([])
        targets = np.array([])

        # Evaluation loop
        if epoch % test_epoch == 0:
            with torch.no_grad():
                model.eval()

                if use_moltree:
                    test_loader = MolTreeFolder(test_folder, vocab, continous=True, batch_size=batch_size, feature_idx=1, label_idx=label_col, num_workers=num_workers, test=True)
                else:
                    test_loader = BaseFolder(test_folder, continous=True, batch_size=batch_size, feature_idx=2, label_idx=label_col, num_workers=num_workers, test=True)
                
                cum_loss = 0
                for (supervised_batch, _) in test_loader:  
                    try:           
                        if len(supervised_batch['labels']) == batch_size:
                            supervised_input = supervised_batch['data']
                            labels = supervised_batch['labels'].cuda() if torch.cuda.is_available() else supervised_batch['labels']
                                
                            loss, clsf_loss, clsf_acc, (pred, target) = model(supervised_input, labels, beta)

                            preds = np.append(preds, pred.cpu().detach().numpy())
                            targets = np.append(targets, target.cpu().detach().numpy())
                                            
                    except Exception as e:
                        traceback.print_exc()
                        continue
                
                print (targets, preds)
                print 'time: %s' % time_since(start)
                print "[Test] Loss: %.3f, Clsf_loss: %.2f" % (loss, clsf_loss)
                
                if cum_loss < best_loss:
                    best_loss = cum_loss
                    no_improvement = 0
                else:
                    no_improvement += 1
                    if no_improvement > patience:
                        print "Ran out of patience, stop"
                        return 
                
                
        if use_moltree:
            train_loader = MolTreeFolder(train_folder, vocab, continous=True, batch_size=batch_size, feature_idx=1, label_idx=label_col, num_workers=num_workers, index_tracker=tracker)
        else:
            train_loader = BaseFolder(train_folder, continous=True, batch_size=batch_size, feature_idx=2, label_idx=label_col, num_workers=num_workers, index_tracker=tracker)
        
        cache = np.zeros(3)
        model.train()

        for (supervised_batch, unsupervised_batch) in train_loader: 
            try:
                if len(supervised_batch['labels']) == batch_size and len(unsupervised_batch['labels']) == batch_size:
                    model.zero_grad()

                    total_step += 1
                    labels = supervised_batch['labels'].cuda() if torch.cuda.is_available() else supervised_batch['labels']
                    
                    unsupervised_loss, _, _, _ = model(unsupervised_batch['data'], None, beta)
                    supervised_loss, clsf_loss, clsf_acc, (pred, target) = model(supervised_batch['data'], labels, beta)
            
                    loss = unsupervised_loss + supervised_loss
                    
                    cache += np.array([loss, clsf_loss, clsf_acc], dtype=np.float)
                    
                    loss.backward()
                    nn.utils.clip_grad_norm_(model.parameters(), clip_norm)
                    optimizer.step()
                    optimizer.zero_grad()

                    if total_step % save_iter == 0:
                        print "Saving model to " + save_dir + "/model.iter-" + "at step: " + str(total_step)
                        torch.save(model.state_dict(), save_dir + "/model.iter-" + str(total_step))

                    if total_step % anneal_iter == 0:
                        scheduler.step()
                        print "learning rate: %.6f" % scheduler.get_lr()[0]

                    if total_step % kl_anneal_iter == 0 and total_step >= warmup:
                        beta = min(max_beta, beta + step_beta)

                    if total_step % print_iter == 0:
                        cache /= print_iter
                        print "Epoch: %d | Iter: %d" % (epoch, total_step)
                        print "[Test] Loss: %.3f, Clsf_loss: %.2f, Clsf_acc: %.2f" % (cache[0], cache[1], cache[2])
                        sys.stdout.flush()
                        cache *= 0

            except Exception as e:
                traceback.print_exc()
                continue

        scheduler.step()

In [ ]:
train(model, optimizer, scheduler)

fast_molvae/qm9_processed/test/tensors-9.pkl opened


Traceback (most recent call last):
Traceback (most recent call last):
  File "fast_jtnn/datautils.py", line 286, in __getitem__
  File "fast_jtnn/datautils.py", line 286, in __getitem__
    'data': tensorize(self.data[idx], self.vocab, assm=self.assm),
    'data': tensorize(self.data[idx], self.vocab, assm=self.assm),
  File "fast_jtnn/datautils.py", line 339, in tensorize
  File "fast_jtnn/datautils.py", line 339, in tensorize
    jtmpn_holder = JTMPN.tensorize(cands, mess_dict)
    jtmpn_holder = JTMPN.tensorize(cands, mess_dict)
  File "fast_jtnn/jtmpn.py", line 122, in tensorize
  File "fast_jtnn/jtmpn.py", line 122, in tensorize
    fatoms = torch.stack(fatoms, 0)
    fatoms = torch.stack(fatoms, 0)
RuntimeError: stack expects a non-empty TensorList
RuntimeError: stack expects a non-empty TensorList
Traceback (most recent call last):
Traceback (most recent call last):
  File "fast_jtnn/datautils.py", line 286, in __getitem__
  File "fast_jtnn/datautils.py", line 286, in __getitem_

Traceback (most recent call last):
  File "fast_jtnn/datautils.py", line 286, in __getitem__
    'data': tensorize(self.data[idx], self.vocab, assm=self.assm),
  File "fast_jtnn/datautils.py", line 286, in __getitem__
  File "fast_jtnn/datautils.py", line 339, in tensorize
    'data': tensorize(self.data[idx], self.vocab, assm=self.assm),
    jtmpn_holder = JTMPN.tensorize(cands, mess_dict)
  File "fast_jtnn/datautils.py", line 339, in tensorize
  File "fast_jtnn/jtmpn.py", line 122, in tensorize
    jtmpn_holder = JTMPN.tensorize(cands, mess_dict)
    fatoms = torch.stack(fatoms, 0)
  File "fast_jtnn/jtmpn.py", line 122, in tensorize
RuntimeError: stack expects a non-empty TensorList
    fatoms = torch.stack(fatoms, 0)
RuntimeError: stack expects a non-empty TensorList
Traceback (most recent call last):
Traceback (most recent call last):
  File "fast_jtnn/datautils.py", line 286, in __getitem__
  File "fast_jtnn/datautils.py", line 286, in __getitem__
    'data': tensorize(self.data[i

    'data': tensorize(self.data[idx], self.vocab, assm=self.assm),
Traceback (most recent call last):
  File "fast_jtnn/datautils.py", line 286, in __getitem__
  File "fast_jtnn/datautils.py", line 339, in tensorize
    jtmpn_holder = JTMPN.tensorize(cands, mess_dict)
    'data': tensorize(self.data[idx], self.vocab, assm=self.assm),
  File "fast_jtnn/datautils.py", line 339, in tensorize
  File "fast_jtnn/jtmpn.py", line 122, in tensorize
    fatoms = torch.stack(fatoms, 0)
    jtmpn_holder = JTMPN.tensorize(cands, mess_dict)
  File "fast_jtnn/jtmpn.py", line 122, in tensorize
RuntimeError: stack expects a non-empty TensorList
    fatoms = torch.stack(fatoms, 0)
RuntimeError: stack expects a non-empty TensorList
Traceback (most recent call last):
Traceback (most recent call last):
  File "fast_jtnn/datautils.py", line 286, in __getitem__
  File "fast_jtnn/datautils.py", line 286, in __getitem__
    'data': tensorize(self.data[idx], self.vocab, assm=self.assm),
    'data': tensorize(se